Copyright 2021 Google LLC.

Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an \"AS IS\" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Dimensions of Motion colab

This brief notebook accompanies code for the paper __Dimensions of Motion: Monocular Prediction through Flow Subspaces__, and may be found at
https://github.com/google-research/google-research/tree/master/dimensions_of_motion.

The project page is at https://dimensions-of-motion.github.io/.

Choose __Run all__ from the Runtime menu to:
* install required packages and download our code,
* compute and visualize a flow basis and reconstruction losses from an example embedding and disparity,
* show how to create a model and compute losses.

Please feel free to contact the authors if you have questions.



# Install packages, download code and example.

In [ ]:
%%shell

# Download code
apt install subversion
svn export --force https://github.com/google-research/google-research/trunk/dimensions_of_motion

# Install required libraries
pip install -r dimensions_of_motion/requirements.txt

# A simple example
mkdir example
cd example
wget https://dimensions-of-motion.github.io/example/source_image.npy
wget https://dimensions-of-motion.github.io/example/target_image.npy
wget https://dimensions-of-motion.github.io/example/flow.npy
wget https://dimensions-of-motion.github.io/example/disparity.npy
wget https://dimensions-of-motion.github.io/example/embedding.npy


# Compute and visualize basis flow fields

In [ ]:
# Make sure python can find our libraries
import sys
sys.path.append('dimensions_of_motion')

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_addons as tfa

import palettable.matplotlib
import flow_vis

import loss


def show_image(data):
  # Just show first image in batch
  data = data[0]
  channels = data.shape[-1]
  if channels == 3:
    plt.imshow(data)
  elif channels == 2:  # flow map
    plt.imshow(flow_vis.flow_to_color(data))
  elif channels == 1:  # disparity
    plt.imshow(data[..., 0],
               cmap=palettable.matplotlib.Viridis_20.mpl_colormap, vmin=0)
  else:
    raise ValueError(f'Bad image data (channels={channels}).')

plt.rcParams["figure.figsize"] = (20,3)

def show_images(names_and_images, images_per_row=4):
  n = len(names_and_images)
  rows = ((n-1) // images_per_row) + 1
  for i in range(n):
    if (i % images_per_row == 0) and i > 0:
      plt.show()
    plt.subplot(1, images_per_row, (i % images_per_row)+1)
    show_image(names_and_images[i][1])
    plt.axis('off')
    plt.title(names_and_images[i][0])
  plt.show()


# Load numpy data and return as a tensor with a batch size of 1.
def load(f):
  return tf.convert_to_tensor(np.load(f))[tf.newaxis]

image = load('example/source_image.npy')
target = load('example/target_image.npy')

flow = load('example/flow.npy')

# Scene representation (output from running model on input image)
disparity = load('example/disparity.npy')
embeddings = load('example/embedding.npy')

losses, summaries = loss.compute_motion_loss(
    image, flow, disparity, embeddings)

show_images([['input', image], ['target', target], ['flow', flow]])

for (key, (value, weight)) in losses.items():
  print(f'{key} = {value[0].numpy(), weight}')

show_images(list(summaries.items()))


# Build example model and compute losses
This codeblock shows how to build a model with 6 embedding dimensions and run
it on an example batch of one image, and uses ground truth flow to compute
losses.


In [ ]:
import models

BATCH_SIZE = 1
WIDTH = 256
HEIGHT = 256
EMBEDDING_DIMENSION = 6

def model():
  session = tf.compat.v1.Session()
  with session.graph.as_default():
    image = tf.compat.v1.placeholder(tf.float32, [BATCH_SIZE, HEIGHT, WIDTH, 3])
    flow = tf.compat.v1.placeholder(tf.float32, [BATCH_SIZE, HEIGHT, WIDTH, 2])
    losses, summaries = models.run_model(image, flow, EMBEDDING_DIMENSION, None)
    session.run(tf.compat.v1.global_variables_initializer())

  def as_numpy(x):
    if tf.is_tensor(x):
      return x.numpy()
    else:
      return x

  def run_model(input_image, input_flow):
    return session.run([losses, summaries], feed_dict={
        image: as_numpy(input_image),
        flow: as_numpy(input_flow)
    })
  return run_model

run_model = model()

print('Running untrained model on input image')
show_images([['image', image]])

losses, summaries = run_model(image, flow)
print('Losses')
for l in losses:
  print('   ', l, losses[l])

print('\nSummary images')
show_images(list(summaries.items()))